# AIエンジニアリング実践2025 第3回 任意課題

## 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get("HF_TOKEN"))

In [3]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import random
random.seed(0)

In [5]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## 質問文の作成

In [6]:
questions = [
    "2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は？",
    "欧州議会がAI法案を採択したのはいつですか？",
    "AIに関する国際連携イニシアティブ（GPAI）はなぜ設立されたのですか？",
    "イギリスではAIの研究開発を推進するためにこれまでにどのような機関・研究所が設立されてきましたか？",
    "近年のAIの研究開発戦略について各国の取り組みを教えて下さい。",
]

## pdfファイルからの文字の抽出

『令和６年版_科学技術・イノベーション白書』の「第1部　第3章　AI関連研究開発の世界の動向」のpdfファイルからテキストを抽出して学習対象とする。

pdfファイルへのリンク（2025/05/05 アクセス）：
<https://www.mext.go.jp/content/20240611-ope_dev03-000036120-4.pdf>

In [7]:
!git clone https://github.com/king-p3nguin/lecture-ai-engineering-day3

Cloning into 'lecture-ai-engineering-day3'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 17.39 KiB | 17.39 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [8]:
with open("/content/lecture-ai-engineering-day3/reference.txt", "r") as f:
  raw_writedown = f.read()

## ベースのLLMでの回答生成（RAGなし）

In [9]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [10]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
responses = []
for question in questions:
    user_prompt =  f"{question}"
    responses.append(generate_output(user_prompt, system_prompt))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpe

In [11]:
for question, response in zip(questions, responses):
    print("質問：", question)
    print("回答：", response)

質問： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は？
回答： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は、NVIDIAのDeep Learning部門である。デミス・ハサビス氏はNVIDIAのCTO（Chief Technology Officer）であり、2010年にNVIDIAのDeep Learning部門を設立した人物です。
質問： 欧州議会がAI法案を採択したのはいつですか？
回答： 欧州議会は、2019年4月16日にAI法案「Regulation on the Liability of Intermediate and Fully Autonomous Artificial Intelligence Systems」を採択しました。
質問： AIに関する国際連携イニシアティブ（GPAI）はなぜ設立されたのですか？
回答： GPAI（Global Partnership on Artificial Intelligence）は、2018年に設立された国際連携イニシアティブです。GPAIは、人工知能（AI）の開発、普及、規律化、安全性、倫理性、平等性など、AIに関する多くの問題に焦点を当てることで、世界中の国々が協力してAIの利益を最大化し、リスクを最小化することを目指しています。

GPAIは、AIの開発や普及に際してのリスクや問題を認識し、世界中の国々が協力してAIの安全性、倫理性、平等性を確保することを目指しています。GPAIは、AIに関する国際的な規律やガイドラインを策定し、AIの開発や普及に際してのリスクや問題を解消することを目指しています。

GPAIの設立の背景には、AIの急速な普及やAIに関するリスクや問題が増加していること、AIの安全性、倫理性、平等性を確保することが重要であることなどがあります。GPA
質問： イギリスではAIの研究開発を推進するためにこれまでにどのような機関・研究所が設立されてきましたか？
回答： イギリスでは、AIの研究開発を推進するために、多くの機関・研究所が設立されてきました。以下は、主要な例です。

1. DeepMind (2010) - AI技術の開発と応用を目的として、Googleが設立した英国支社。
2. The Alan Turing Instit

## RAGを組み合わせた回答生成

### 初期RAG

* **ドキュメント処理**:
  - pypdfを用いてpdfから抽出したテキストを人手で丁寧に修正
  - 各ページごとに記載されている参考文献と番号を削除

* **検索手法**:
  - 「。」（句点）で区切られた文単位でテキストを分割
  - シンプルな類似度ベースの検索でクエリに関連する文を抽出
  - クエリに関連する文とその前後2文を参考文献として入力

In [12]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [13]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[20])

ドキュメントサイズ:  132
ドキュメントの例: 
 また、国防高等研究計画局（ＤＡＲＰＡ）は、2018年、次世代ＡＩ技術の開発のため、「AI NEXT Campaign」と称した取組を、複数年にわたり総額20億ドル以上を投じて進めることを発表しています


In [14]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
topk = 10

responses = []
for question in questions:
    print("質問：", question, "\n\n")
    # Retrievalの実行
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)

    # 各ドキュメントの類似度スコア
    scores = (query_embeddings @ document_embeddings.T) * 100
    for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
        print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
        print(documents[index], "\n")

    # 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
    references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
    user_prompt =  f"[参考資料]\n{references}\n\n[質問]\n{question}"
    responses.append(generate_output(user_prompt, system_prompt))

質問： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は？ 




/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


取得したドキュメント1: (Score: 82.99681091308594)
また、2010年にデミス・ハサビス氏により設立されたスタートアップのDeepMind社が2014年にGoogle社に買収されたことや、国際的な機械学習への関心の高まりも受け、アラン・チューリング研究所を中心としたＡＩの研究が拡充されるとともに、2019年には国民保健サービス（ＮＨＳ）を担当するＮＨＳＸ（当時）の中にNHS AI Labを設置することが発表されるなど、政府機関による国民サービスにおけるＡＩの活用も進められてきています 

取得したドキュメント2: (Score: 80.38186645507812)
2012年にはヒントン教授（当時）が率いるトロント大学のチームが、「深層畳み込みニューラルネットワーク」を用いて、高い精度の画像認識技術を示して世界的な注目を集め、ヒントン教授（当時）が起業したスタートアップ企業DNNresearch社が2013年にGoogle社に買収されたことなどから、トロントやモントリオールなどに大手テクノロジー企業が研究開発拠点を設置し、またＡＩ関連のスタートアップ企業も集積してきています 

取得したドキュメント3: (Score: 79.07408142089844)
産業界では、Google社、Microsoft社、Amazon社などのＩＴ大手企業が、ＡＩをコア技術として、様々な事業を展開しています 

取得したドキュメント4: (Score: 78.37621307373047)
またそのような取組を通じて、1,100ものＡＩスタートアップ企業がシンガポールを本拠地に選ぶなど、ＡＩエコシステムの形成が進められてきました 

取得したドキュメント5: (Score: 77.44464874267578)
例えばGoogle社は、ＡＩを活用した検索、翻訳、音声認識といったソフトのサービスのみならず、対話型生成ＡＩである「Gemini（旧Bard）」や、ＡＩを活用した自動運転技術の開発などにも取り組んでいます 

取得したドキュメント6: (Score: 77.03466796875)
また、第１章で見てきたような高度ＡＩ技術の急速な進展と関心の高まりを受けて、米国のバイデン政権は、2023年５月に責任あるイノベーションのための取組方針を示すとともに、

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


取得したドキュメント1: (Score: 86.15695190429688)
このうち、法的な枠組みの確保に関しては、欧州委員会は、2021年４月に、「ＡＩ法案」を提案し、その後の議論により修正が重ねられた上で、2024年３月、欧州議会で、同法案が採択されました 

取得したドキュメント2: (Score: 79.10029602050781)
そのような中、2018年、「AI for Europe」と題した戦略が策定され、欧州の技術的・産業的能力の強化や、倫理的・法的枠組みの確保などの方向性が示されました 

取得したドキュメント3: (Score: 77.54623413085938)
英国議会では、上院通信・デジタル委員会がＬＬＭと生成ＡＩに関する調査を行い、2024年２月に報告書を発表するとともに、ＡＩに関する議員提出法案についての議論も行われています 

取得したドキュメント4: (Score: 76.68551635742188)
● 欧州評議会の「ＡＩ、人権、民主主義、法の支配に関する枠組条約」
人権、民主主義、法の支配の分野で国際社会の基準策定を主導する汎欧州の国際機関で、日本、米国、カナダ、メキシコ等もオブザーバー国として参加している欧州評議会（Council of Europe）においても、「ＡＩ、人権、民主主義、法の支配に関する枠組条約」について議論が進められています 

取得したドキュメント5: (Score: 76.1866455078125)
同法案は、ＡＩの安全性と倫理を確保し、ＡＩがＥＵの価値観に沿った形で開発・利用されるようにすることを目的とし、ＡＩの開発・利用に適用される一般原則を定めるとともに、特定のリスクを伴うＡＩシステムの開発・利用に規制を適用することとしています 

取得したドキュメント6: (Score: 76.0162353515625)
また、欧州フレームワークの中で、機械翻訳を中心に研究開発に対しても支援がなされてきているとともに、2024年１月には、欧州委員会は、ＡＩイノベーションの支援に向けた政策パッケージを発表し、スタートアップや中小企業などを対象とした支援の枠組みを提示しています 

取得したドキュメント7: (Score: 75.98883819580078)
１－３．欧州連合（ＥＵ）
ＥＵでは、データ

...

取得したドキュメント1: (Score: 86.08161163330078)
● ＡＩに関する国際連携イニシアティブ（ＧＰＡＩ）
2020年（令和２年）に、人権と民主的価値を尊重するようなＡＩの活用を推進する国際連携のイニシアティブとして、ＧＰＡＩが設立されました 

取得したドキュメント2: (Score: 83.46685028076172)
ＧＰＡＩは、人間中心の考えに立ち「責任あるＡＩ」の開発・利用を実現するために設立された国際官民連携組織であり、その設立は、2019年（令和元年）８月にフランスで開催されたＧ７ビアリッツサミットにおいて、必要性が提唱されたことに端を発します 

取得したドキュメント3: (Score: 80.21990203857422)
これを受け、2020年（令和２年）５月に開催されたＧ７科学技術大臣会合において、ＧＰＡＩの設立に向けたＧ７の協力が合意に至りました 

取得したドキュメント4: (Score: 78.89288330078125)
ＧＰＡＩには、2024年（令和６年）３月現在で29の国・地域が参加しており、各国はＡＩの開発と利用において民主主義、人権、包摂、多様性、イノベーションなどの価値観を共有しています 

取得したドキュメント5: (Score: 78.04752349853516)
その後、2020年（令和２年）６月にＧＰＡＩの設立宣言が採択されて、正式に設立されました 

取得したドキュメント6: (Score: 77.06013488769531)
またＧＰＡＩは、ＡＩの責任ある開発・利用を実現するための取組を進めており、責任あるＡＩ、データ・ガバナンス、仕事の未来、イノベーションと商業化の四つのテーマについて専門家のワーキンググループを設置し、プロジェクトを実施しています 

取得したドキュメント7: (Score: 76.98876953125)
今後も、ＡＩの健全な発展と社会への利活用を促進するために、ＧＰＡＩの活動が期待されます 

取得したドキュメント8: (Score: 75.98635864257812)
ＡＩが人類のより大きな利益のために活用されることを目指す国連の取組を支援する目的で、国連事務総長の主導の下、国連ハイレベルＡＩ諮問機関が2023年10月に設立されました 

取得したドキュメ

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


取得したドキュメント1: (Score: 81.9822769165039)
そのような強みを生かして、英国政府では、ＡＩを安全で信頼できる技術として発展させ、経済成長と社会の持続的発展への寄与を目指すという方針の下、特に2010年代以降、研究開発や産学官の連携、データ共有等について、積極的に様々な政策を講じてきています 

取得したドキュメント2: (Score: 81.91893005371094)
そして、2023年３月に発表した「UK Science and Technology Framework」の中で、ＡＩを五つの重要科学技術の一つと位置付けるとともに、ＡＩ分野の研究者の育成・支援政策を発表し、英国研究・イノベーション機構（ＵＫＲＩ）を通じて、責任ある信頼できるＡＩの確立を目指した研究や人材育成を行う大学に対して支援が行われています（第１-３-５表） 

取得したドキュメント3: (Score: 81.44972229003906)
また、英国政府は、ＡＩ安全研究所（ＵＫＡＩＳＩ）の設置を発表し、今後、国際パートナーとも連携しながら、最先端ＡＩの安全性を検証していくこととなっています 

取得したドキュメント4: (Score: 80.92586517333984)
１－２．英国
英国では、1950年に数学者アラン・チューリング博士により発表された論文がその後のＡＩ研究の起源ともなっているように、大学を中心にＡＩに関する基礎研究から応用研究まで幅広く研究開発が行われてきています 

取得したドキュメント5: (Score: 80.8934326171875)
また、2010年にデミス・ハサビス氏により設立されたスタートアップのDeepMind社が2014年にGoogle社に買収されたことや、国際的な機械学習への関心の高まりも受け、アラン・チューリング研究所を中心としたＡＩの研究が拡充されるとともに、2019年には国民保健サービス（ＮＨＳ）を担当するＮＨＳＸ（当時）の中にNHS AI Labを設置することが発表されるなど、政府機関による国民サービスにおけるＡＩの活用も進められてきています 

取得したドキュメント6: (Score: 80.14253997802734)
オックスフォード大学には、2015年に「戦略的人工知能研究センター（ＳＡＩＲＣ）」が

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


取得したドキュメント1: (Score: 84.51042938232422)
このような中、各国では、ＡＩ技術に携わる人材の育成や、ＡＩの利活用に係るルールなどを整備しながら、ＡＩに関する国家戦略を策定し、研究開発を推進しています 

取得したドキュメント2: (Score: 83.98136901855469)
第１節  主要国・地域におけるＡＩに関する研究開発戦略

１－１．米国
米国政府は、ＡＩ分野の研究開発を戦略的に進めるため、2016年に「国家ＡＩ研究開発戦略（National AI R&D Strategic Plan）」を策定し、ＡＩ研究への長期的な投資、人間とＡＩの協調、倫理的・法的・社会的影響の理解等に重点的に取り組む方針を示しました 

取得したドキュメント3: (Score: 83.54259490966797)
本章では、ＡＩの研究開発を進める主要な国・地域の動向を紹介するとともに、国際的な議論や多国間の連携・協働についても説明します 

取得したドキュメント4: (Score: 83.09654235839844)
このような優れた研究や人材を土台として、カナダ政府は、2017年３月、「汎カナダＡＩ戦略」と題した国家戦略を発表し、さらに2022年６月には同戦略の第２段階の開始を発表して、①商用化、②標準、③人材育成と研究開発の三つの柱に沿って、様々な取組が展開されています 

取得したドキュメント5: (Score: 82.47760009765625)
具体的には、2018年11月に「国家ＡＩ戦略」を発表し、ＡＩを安全で信頼できる技術として発展させ、フランスの経済成長と社会の持続的発展に貢献することを目指して、国立情報学自動制御研究所（Ｉｎｒｉａ）が主導するＡＩプログラムの全国展開や、フランス国立研究機構（ＡＮＲ）におけるＡＩ研究の強化、ＡＩ人材育成、計算資源の増強、二国間・欧州域内・国際協力の強化等の取組に対して、2022年までの４年間に総額６億6,500万ユーロを措置することが計画されました 

取得したドキュメント6: (Score: 82.33635711669922)
１－４．ドイツ
ドイツは2018年に「ＡＩ戦略（Nationale Strategie für Künstliche Intelligenz）」を発表し、ドイツ

In [15]:
for question, response in zip(questions, responses):
    print("質問：", question)
    print("回答：", response)

質問： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は？
回答： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は、DeepMind社です。DeepMind社は、2014年にGoogle社に買収されました。
質問： 欧州議会がAI法案を採択したのはいつですか？
回答： 欧州議会がAI法案を採択したのは、2024年3月です。
質問： AIに関する国際連携イニシアティブ（GPAI）はなぜ設立されたのですか？
回答： AIに関する国際連携イニシアティブ（GPAI）は、2019年8月にフランスで開催されたG7ビアリッツサミットにおいて、必要性が提唱されたことに端を発します。同サミットでは、AIの開発と利用において「人間中心の価値」を尊重し、AIの潜在的なリスクを軽減するための国際的な協力の必要性が確認されました。

その後、2020年5月に開催されたG7科学技術大臣会合において、GPAIの設立に向けたG7の協力が合意に至りました。2020年6月には、GPAIの設立宣言が採択され、正式に設立されました。

GPAIは、人間中心の考えに立ち「責任あるAI」の開発・利用を実現するために設立された国際官民連携組織です。GPAIには、現在29の国・地域が参加しており、各国はAIの開発と利用において民主主義、人権、包摂、多様性、イノベーションなどの価値観を共有
質問： イギリスではAIの研究開発を推進するためにこれまでにどのような機関・研究所が設立されてきましたか？
回答： イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。

* アラン・チューリング研究所（Alan Turing Institute）：2015年に設立された国立の研究機関で、データ・サイエンスの研究を行う。
* NHS AI Lab：2019年に設立された国民保健サービス（NHS）を担当する機関で、AIを活用した医療サービス開発に取り組む。
* SAIIRC（戦略的人工知能研究センター）：2015年に設立されたオックスフォード大学の研究所で、AIに関する研究開発を行う。
* Leverhulme研究所：2016年に設立されたケンブリッジ大学の研究所で、AIに関する研究開発を行う。
* AI安全研究所（UK AI Safety In

### 修正RAG

* **ドキュメント処理**:
  - pypdfを用いてpdfから抽出したテキストを人手で丁寧に修正
  - 各ページごとに記載されている参考文献と番号を削除

* **検索手法**:
  - 段落単位でテキストを分割
  - シンプルな類似度ベースの検索でクエリに関連する段落を抽出
  - クエリに関連する段落を参考文献として入力

In [34]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.splitlines() if text.strip() != ""]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[8])

ドキュメントサイズ:  44
ドキュメントの例: 
 ＥＵでは、データガバナンスと規制に重点を置いた議論が進められるとともに、ＡＩの開発と利用に関する環境整備を進めています。また、第４章に示すとおり科学におけるＡＩの活用についての議論も進められています。まず、欧州では、データの安全性とプライバシーを保護するための規制である「一般データ保護規則（ＧＤＰＲ）」が2018年に施行されました。ＧＤＰＲは、ＥＵ域内で個人データを収集・利用する企業に対して、データ主体の同意取得、データの安全性確保、データの移転制限などの義務を課しています。その際には、ＡＩシステムの開発や利用においても、ＥＵ域内の個人データを収集・利用する場合には、ＧＤＰＲの遵守が求められます。そのような中、2018年、「AI for Europe」と題した戦略が策定され、欧州の技術的・産業的能力の強化や、倫理的・法的枠組みの確保などの方向性が示されました。このうち、法的な枠組みの確保に関しては、欧州委員会は、2021年４月に、「ＡＩ法案」を提案し、その後の議論により修正が重ねられた上で、2024年３月、欧州議会で、同法案が採択されました。同法案は、ＡＩの安全性と倫理を確保し、ＡＩがＥＵの価値観に沿った形で開発・利用されるようにすることを目的とし、ＡＩの開発・利用に適用される一般原則を定めるとともに、特定のリスクを伴うＡＩシステムの開発・利用に規制を適用することとしています。具体的には、ＡＩシステムの開発、使用、運用に際して遵守すべき義務を定めており、ＡＩシステムを４段階のリスクレベル（許容できないリスク、高リスク、限定的なリスク、最小限のリスク）に分類（第１-３-６図）し、リスクレベルに応じた規制及び罰則を適用するというリスクベースのアプローチを導入しています。なお、本法案はＥＵ加盟国に所在する事業者のみならず、ＥＵ域内において高リスクのＡＩサービスを提供しようとしたり、市場に参入しようとしたりする第三国の事業者に対しても適用されます。さらに、汎用目的ＡＩ（ＧＰＡＩ）システムとそのベースとなるＧＰＡＩモデルに関しては、重大な汎用性を示しながらも、広範囲の明確なタスクを適切に実行でき、かつ様々な下流のシステムやアプリケーションに統合できるＡＩモデルと位置付けられることから、よりシステミックなリスク（システム全体

In [44]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
topk = 5

responses = []
for question in questions:
    print("質問：", question, "\n\n")

    references = []
    for paragraph in documents:
        if len(paragraph) > 30: # 章タイトルなどを除外
            filter_system_prompt = "与えられた参考資料から質問に直接関連する情報のみを出力してください。必ず「日本語で回答」すること。余計な情報は出力しないでください。"
            filter_question =  f"[参考資料]\n{paragraph}\n\n[質問]\n{question}"
            filter_response = generate_output(filter_question, filter_system_prompt)

            references.append(filter_response)

    references = "\n".join(references)
    new_documents = [text.strip() for text in references.splitlines() if text.strip() != ""]

    # Retrievalの実行
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(new_documents)

    # 各ドキュメントの類似度スコア
    scores = (query_embeddings @ document_embeddings.T) * 100
    for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
        print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
        print(new_documents[index], "\n")

    # 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
    references = "\n".join(["* " + "。".join(new_documents[max(0, i-2): min(i+2, len(new_documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
    user_prompt =  f"[参考資料]\n{references}\n\n[質問]\n{question}"
    responses.append(generate_output(user_prompt, system_prompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


質問： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は？ 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

取得したドキュメント1: (Score: 93.4949722290039)
参考資料には、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は存在しません。質問に直接関連する情報は出力しません。 

取得したドキュメント2: (Score: 93.25103759765625)
参考資料には、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は存在しません。質問に直接関連する情報は出力されません。 

取得したドキュメント3: (Score: 93.0721664428711)
参考資料には、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は存在しません。質問に直接関連する情報は、存在しません。 

取得したドキュメント4: (Score: 92.65911865234375)
参考資料には、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は存在しません。 

取得したドキュメント5: (Score: 92.65911865234375)
参考資料には、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は存在しません。 



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

取得したドキュメント1: (Score: 92.32192993164062)
イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。 

取得したドキュメント2: (Score: 92.32192993164062)
イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。 

取得したドキュメント3: (Score: 92.32192993164062)
イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。 

取得したドキュメント4: (Score: 92.32192993164062)
イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。 

取得したドキュメント5: (Score: 92.32192993164062)
イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。 



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


質問： 近年のAIの研究開発戦略について各国の取り組みを教えて下さい。 




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

取得したドキュメント1: (Score: 91.95684051513672)
近年のAIの研究開発戦略について、各国の取り組みとして、以下のようなものがある。 

取得したドキュメント2: (Score: 91.59358978271484)
参考資料に基づいて、以下は各国のAIの研究開発戦略についての情報です。 

取得したドキュメント3: (Score: 88.37769317626953)
以上のように、各国はAIの研究開発を推進するために、各種の戦略や計画を立てている。 

取得したドキュメント4: (Score: 87.5746078491211)
以上のように、各国のAI研究開発戦略は、AIの研究開発を促進するために、各国の政府や企業が取り組むものです。 

取得したドキュメント5: (Score: 85.15310668945312)
OECDの報告書では、G7の共通理解に向けたAIに関する報告書において、各国のAI研究開発戦略について以下のように述べている。 



In [45]:
for question, response in zip(questions, responses):
    print("質問：", question)
    print("回答：", response)

質問： 2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業は？
回答： 参考資料には、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は存在しません。したがって、2010年にデミス・ハサビス氏により設立されたAIスタートアップ企業に関する情報は出力されません。
質問： 欧州議会がAI法案を採択したのはいつですか？
回答： 日本語で回答します。

参考資料には、欧州議会がAI法案を採択した日付に関する情報は含まれていません。ただし、欧州議会がAI法案を採択したのは、2020年10月21日であることが明確です。
質問： AIに関する国際連携イニシアティブ（GPAI）はなぜ設立されたのですか？
回答： AIに関する国際連携イニシアティブ（GPAI）は、2023年10月に国連事務総長の主導の下に設立されました。GPAIの目的は、ＡＩが人類のより大きな利益のために活用されることを目指す国連の取組を支援することです。GPAIは、AI技術の開発や普及に対する国際的な協力体制を構築し、AIの利活用や倫理的・法的規制の整備を目指すものです。
質問： イギリスではAIの研究開発を推進するためにこれまでにどのような機関・研究所が設立されてきましたか？
回答： イギリスでは、AIの研究開発を推進するために、以下のような機関・研究所が設立されてきました。

* The Alan Turing Institute (ATI)：2016年に設立されたAI研究機関。AIの研究開発、教育、イノベーションを推進することを目的としています。

また、2018年に設立されたThe Ada Lovelace Instituteは、AIの倫理的・社会的影響の研究、AIの開発のための倫理的ガイドラインの作成などを目的としています。

以上のように、イギリスではAIの研究開発を推進するために、多くの機関・研究所が設立されてきています。
質問： 近年のAIの研究開発戦略について各国の取り組みを教えて下さい。
回答： 近年のAIの研究開発戦略について、各国の取り組みは以下の通りです。

* アメリカ：米国政府は、2016年に「National Artificial Intelligence Research and Development Strategic Plan